In [4]:
import os
if "master" not in os.listdir("./"):
  !git clone https://github.com/YounesHB92/master.git
  !pip install -r ./master/requirements.txt
else:
  print("Already cloned!")

Cloning into 'master'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 118 (delta 17), reused 105 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 948.38 KiB | 27.89 MiB/s, done.
Resolving deltas: 100% (17/17), done.
Filtering content: 100% (6/6), 962.13 MiB | 41.50 MiB/s, done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: './content/master/requirements.txt'


In [2]:
import sys
if "/content/master" not in sys.path:
  sys.path.append('/content/master')
else:
  print("Already added!")

In [3]:
if "datasets.tar.gz" not in os.listdir("./"):
  !gdown "https://drive.google.com/uc?id=1Gtw4q4pkMHWMYz5urew5C44X17oLE9e3"
  !tar -xvf datasets.tar.gz -C ./master
else:
  print("Already downloaded!")

Already downloaded!


In [6]:
from src.datasets import DatasetIterator
from src.training import Trainer
from src.models import LoadModel
from src.utils import Optimizer

In [9]:
encoders = [
    "tu-mvitv2_base",
    "swin_base_patch4_window7_224",
    "convnext_base"
]

main_device = "colab" if "google.colab" in str(get_ipython()) else "local"
output_path = "/content/drive/MyDrive/train_outputs" if main_device == "colab" else "./experiments/outputs"
datasets_base_path = "./master/datasets" if main_device == "colab" else "./datasets"

for encoder_name in encoders:
    dataset_loader_ = DatasetIterator(
        train_path=os.path.join(datasets_base_path, "splits/masks/train"),
        val_path=os.path.join(datasets_base_path, "splits/masks/val"),
        batch_size=4,
        augment=True
    )
    model_ = LoadModel(
        encoder=encoder_name,
        num_classes=dataset_loader_.num_classes
    )
    optimizer_ = Optimizer(
        model=model_.model
    )

    trainer_ = Trainer(
        model=model_.model,
        train_loader=dataset_loader_.train_loader,
        val_loader=dataset_loader_.val_loader,
        loss_function=optimizer_.loss_function,
        optimizer=optimizer_.optimizer,
        scheduler=optimizer_.scheduler,
        device="cuda",
        output_dir=output_path,
        num_epochs=2,
        details="test in local computer where augmentation is applied",
        augment=dataset_loader_.augment
    ).run_training()

Datasets are ready!
	Detected 7 classes: 
		b
l
o
c
k
	
c
r
e
s
c
e
n
t
	
c
r
o
c
o
d
i
l
e
	
d
i
a
g
o
n
a
l
	
l
o
n
g
i
t
u
d
i
n
a
l
	
m
e
a
n
d
e
r
i
n
g
	
t
r
a
n
s
v
e
r
s
e
datasets_.train_loader and datasets_.val_loader to be used!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/206M [00:00<?, ?B/s]

ClassificationHead(
  (encoder): TimmUniversalEncoder(
    (model): FeatureGetterNet(
      (model): MultiScaleVit(
        (patch_embed): PatchEmbed(
          (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        )
        (stages): ModuleList(
          (0): MultiScaleVitStage(
            (blocks): ModuleList(
              (0-1): 2 x MultiScaleBlock(
                (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
                (attn): MultiScaleAttention(
                  (qkv): Linear(in_features=96, out_features=288, bias=True)
                  (proj): Linear(in_features=96, out_features=96, bias=True)
                  (pool_q): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96, bias=False)
                  (norm_q): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
                  (pool_k): Conv2d(96, 96, kernel_size=(3, 3), stride=(4, 4), padding=(1, 1), groups=96, bias=False)
                  (norm

  0%|          | 0/175 [00:00<?, ?it/s]


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx